In [47]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt -e https_proxy=http://daicelproxy3:80
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt -e https_proxy=http://daicelproxy3:80

File 'edgar_allan_poe.txt' already there; not retrieving.

File 'robert_frost.txt' already there; not retrieving.



In [48]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [49]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

In [50]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [51]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [90]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
    print(f"{f} corresponds to label {label}")

    for line in open(f):
        line = line.rstrip().lower()
        if line:
            # remove punctuation
            line = line.translate(str.maketrans("","", string.punctuation))
            input_texts.append(line)
            labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [91]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [92]:
len(Ytrain), len(Ytest)

(1615, 539)

In [93]:
train_text[:5]

['shall dawn upon him desolate',
 'what do we see in such a hole i wonder',
 'what childishness he wouldnt give way to',
 'it makes it all the worse you must be blind',
 'waiting for things to happen in their favour']

In [94]:
test_text[:5]

['only bewitched so i would last him longer',
 'and a trapper looking in at the door',
 'who alterest all things with thy peering eyes',
 'she was shut in for life she lived her whole',
 'on fathers side it seems were let me see  ']

In [96]:
idx = 1
word2idx = {"<unk>":0}

In [97]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx
            idx += 1

In [98]:
len(word2idx)

2523

In [99]:
# convert data into integer format

train_text_int = []
test_text_int = []

for text in train_text:
    tokens = text.split()
    line_as_int = [word2idx[token] for token in tokens]
    train_text_int.append(line_as_int)

for text in test_text:
    tokens = text.split()
    line_as_int = [word2idx.get(token, 0) for token in tokens]
    test_text_int.append(line_as_int)

In [100]:
train_text_int[100:105]

[[390, 25, 391, 93, 35, 153],
 [392, 25, 393, 47, 49, 394, 69, 180],
 [102, 25, 395, 69, 396, 105],
 [69, 14, 397, 12, 398, 399, 90, 400],
 [14, 401, 21, 402, 27, 21, 25, 403]]

In [101]:
test_text_int[100:105]

[[174, 45, 108, 63, 0, 47, 1859, 0],
 [50, 1135, 664, 47, 24, 29, 0],
 [72, 350, 1130, 2465],
 [583, 334, 664, 0, 0, 0],
 [465, 0, 81, 25, 0, 420, 0]]

In [103]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [104]:
# compute counts for A and pi

def compute_counts(text_as_int, A, pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            if last_idx is None:
                # its the first word in a sentence
                pi[idx] += 1
            else:
                # the last word exists, so count a transition
                A[last_idx, idx] += 1
            
            # update last idx
            last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [105]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before

A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [109]:
# log A and pi since we dont need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [110]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33436532507739936, 0.6656346749226006)

In [111]:
# build the classifier
class Classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) # number of classes

    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                #its the first token
                logprob += logpi[idx]
            else:
                logprob += logA[last_idx, idx]

            # update last_idx
            last_idx = idx
        
        return logprob
    
    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.K)]
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions

In [112]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [113]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Train acc: 0.9956656346749226
Test acc: 0.8200371057513914


In [114]:
from sklearn.metrics import confusion_matrix, f1_score

In [115]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 533,    7],
       [   0, 1075]], dtype=int64)

In [116]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 91,  87],
       [ 10, 351]], dtype=int64)

In [119]:
print(f"Train f1: {f1_score(Ytrain, Ptrain)}")
print(f"Test f1: {f1_score(Ytest, Ptest)}")

Train f1: 0.9967547519703291
Test f1: 0.8785982478097621
